In [1]:
import torch
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from transformers import (AutoTokenizer,
                          T5ForConditionalGeneration,
                          GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification)

device = torch.device('cuda' if torch.cuda.is_available() else 
                      ('mps' if torch.backends.mps.is_available() else 'cpu'))

import warnings
warnings.simplefilter("ignore", UserWarning)

### Summarizing

In [2]:
def upload_summary_model():
    summary_model_name = "IlyaGusev/rut5_base_sum_gazeta"
    summary_tokenizer = AutoTokenizer.from_pretrained(summary_model_name)
    summary_model = T5ForConditionalGeneration.from_pretrained(summary_model_name)
    return summary_tokenizer, summary_model

In [3]:
def summarize(text):
    input_ids = summary_tokenizer(
    text,
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
    )["input_ids"]

    output_ids = summary_model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]
    
    summary = summary_tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [4]:
summary_tokenizer, summary_model = upload_summary_model()

In [6]:
summarize('Добрый день. Подскажите пожалуйста по такому вопросу. Подала на выплату с 3-7. Есть старший сын студент очник. Предоставила в соц защиту справку о стипендии в электронном варианте. Соц.защита требует в бумажном виде . Сказали , что будет отказ. Дети на практике и взять справку в Яндексе на руки не предоставляется возможным. В прошлом году сама соц.защита предложила в электронном варианте справку послать, а в этом нужна бумага. Правомерно ли поступают в нашей соц.защите г. Чусового? Спасибо.')

'Подала на выплату стипендии с 3-7. Есть старший сын студент очник, а его старший сын очник. В соц.защите требуют в бумажном виде справку о стипендии в электронном варианте, а в этом нужна бумага.'

### ruGPT-3 large and small labels classification

In [7]:
def upload_gpt_large_model():
    model_path = 'ai-forever/rugpt3small_based_on_gpt2'
    large_label_model_path = './models/rugpt_large_labels_84.pt'
    max_length = 612
    
    model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_path, num_labels=26)
    
    large_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_path)
    large_tokenizer.padding_side = "left"
    large_tokenizer.pad_token = large_tokenizer.eos_token

    large_label_model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_path, 
                                                                      config=model_config)

    large_label_model.resize_token_embeddings(len(large_tokenizer))
    large_label_model.config.pad_token_id = large_label_model.config.eos_token_id

    large_label_model.load_state_dict(torch.load(large_label_model_path, map_location=torch.device(device)))
    large_label_model.to(device)
    
    return large_tokenizer, large_label_model
    

In [8]:
def upload_gpt_small_model():
    model_path = 'ai-forever/rugpt3small_based_on_gpt2'
    small_label_model_path = './models/rugpt_small_labels_74.pt'
    max_length = 612
    
    model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_path, num_labels=195)
    
    small_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_path)
    small_tokenizer.padding_side = "left"
    small_tokenizer.pad_token = small_tokenizer.eos_token

    small_label_model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_path, 
                                                                      config=model_config)

    small_label_model.resize_token_embeddings(len(small_tokenizer))
    small_label_model.config.pad_token_id = small_label_model.config.eos_token_id

    small_label_model.load_state_dict(torch.load(small_label_model_path, map_location=torch.device(device)))
    small_label_model.to(device)
    
    return small_tokenizer, small_label_model
    

In [9]:
large_tokenizer, large_label_model = upload_gpt_large_model()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ai-forever/rugpt3small_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
small_tokenizer, small_label_model = upload_gpt_small_model()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ai-forever/rugpt3small_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def predict_class(text, tokenizer, model, classes):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    return classes[predicted_class], probabilities[0][predicted_class].item()

In [16]:
classes_large = {0: 'Благоустройство',
                 1: 'Социальное обслуживание и защита',
                 2: 'Общественный транспорт',
                 3: 'Здравоохранение/Медицина',
                 4: 'Мусор/Свалки/ТКО',
                 5: 'Образование',
                 6: 'Дороги',
                 7: 'ЖКХ',
                 8: 'Коронавирус',
                 9: 'Экономика и бизнес',
                 10: 'Культура',
                 11: 'Связь и телевидение',
                 12: 'Газ и топливо',
                 13: 'Безопасность',
                 14: 'Спецпроекты',
                 15: 'Мобилизация',
                 16: 'МФЦ "Мои документы"',
                 17: 'Физическая культура и спорт',
                 18: 'Торговля',
                 19: 'Строительство и архитектура',
                 20: 'Памятники и объекты культурного наследия',
                 21: 'Экология',
                 22: 'Государственная собственность',
                 23: 'Роспотребнадзор',
                 24: 'Погребение и похоронное дело',
                 25: 'Электроснабжение'}

In [17]:
classes_small = {0: '★ Ямы во дворах',
                 1: 'Оказание гос. соц. помощи',
                 2: 'Дети и многодетные семьи',
                 3: 'Содержание остановок',
                 4: 'Технические проблемы с записью на прием к врачу',
                 5: 'Плата за вывоз ТКО',
                 6: 'Образовательный процесс',
                 7: 'Диспансеризация',
                 8: '★ Просьбы о лечении',
                 9: '★ Нарушение правил очистки дорог от снега и наледи/Обращения о необходимости очистить тротуар от снега и наледи',
                 10: 'Предложение по благоустройству',
                 11: 'Задержка выплат гражданам',
                 12: 'Аварийное жилье/переселение',
                 13: '★ Нарушение правил уборки от снега и наледи внутридворового проезда, тротуара, площади',
                 14: 'Жалобы на управляющие компании',
                 15: 'Борщевик Сосновского',
                 16: 'Пешеходные переходы и жд переезды',
                 17: '★ Уборка/Вывоз мусора',
                 18: 'Жалобы на врачей',
                 19: 'Трудоустройство',
                 20: 'Подтопление автомобильных дорог',
                 21: 'Тесты на коронавирус',
                 22: '★ Оказание медицинской помощи не в полном объеме/отказ в оказании медицинской помощи',
                 23: 'Парки и зоны отдыха',
                 24: 'Содержание больниц',
                 25: 'Уборка территорий',
                 26: '★ Отсутствие контейнерной площадки/Проезд к контейнерной площадке',
                 27: 'Благоустройство общественного пространства (парк, сквер, пешеходная зона, бульвар, набережная, центральная улица или площадь)',
                 28: '★ Питание в медицинских учреждениях',
                 29: 'Нехватка или сокращение врачей и медицинских учреждений',
                 30: 'Необходима установка и замена дорожных ограждений',
                 31: '★ Ненадлежащее содержание зеленых насаждений (газонов)/деревьев и кустарников',
                 32: 'Вырубка деревьев, кустарников',
                 33: 'Предложить установку нового лежачего полицейского (ИДН)',
                 34: 'Цены и ценообразование',
                 35: 'Учреждения культуры',
                 36: 'Профильный осмотр',
                 37: 'Доступность вакцин',
                 38: 'Плата за ЖКУ и работа ЕИРЦ',
                 39: 'Культурно-досуговые мероприятия',
                 40: 'Отсутствие холодной воды',
                 41: 'Ремонт/строительство мостов',
                 42: 'Завышение платы за коммунальные услуги',
                 43: 'Очередь',
                 44: 'Ненадлежащее качество или отсутствие отопления',
                 45: 'Организация парковок',
                 46: 'Ненадлежащая уборка подъездов и лифтов',
                 47: 'Коронавирус',
                 48: 'Неудовлетворительные условия проезда в транспорте на действующем маршруте',
                 49: 'Содержание, ремонт и обустройство тротуаров',
                 50: 'Отсутствие лекарств в аптеках',
                 51: 'Порядок и пункты вакцинации',
                 52: '★ Информационно-техническая поддержка',
                 53: 'Восстановление газоснабжения',
                 54: 'Отсутствие горячей воды',
                 55: 'Некачественный капитальный ремонт',
                 56: 'Ошибки врачей, халатность',
                 57: 'Создание доступной среды для инвалидов',
                 58: '★ Неисправные фонари освещения',
                 59: '★ Скорая помощь',
                 60: 'Сборы за капитальный ремонт',
                 61: '★ Открытые канализационные люки',
                 62: 'Социальные учреждения',
                 63: 'Безопасность общественных пространств',
                 64: 'Льготные лекарства',
                 65: 'Проблемы с контейнерами',
                 66: 'Ямы и выбоины на дороге',
                 67: 'Дезинфекция МКД',
                 68: 'Вакцинация',
                 69: '★ Затопление подъездов, подвальных помещений',
                 70: 'Строительство или реконструкция дорог',
                 71: 'Отсутствие фонарей освещения',
                 72: 'Ремонт дороги',
                 73: 'Занятость и трудоустройство',
                 74: '★ Некачественно нанесенная разметка на проезжей части',
                 75: 'Разрушение тротуаров и пешеходных дорожек',
                 76: 'Отсутствие детских площадок',
                 77: 'Ливневые канализации (строительство и реконструкция)',
                 78: 'Отлов безнадзорных собак и кошек',
                 79: 'Некорректное поведение водительского состава',
                 80: 'Плохое качество воды',
                 81: 'Спецпроекты',
                 82: 'Льготы на проезд и тарифы',
                 83: 'Поддержка семей мобилизованных',
                 84: 'Инвалиды',
                 85: '★ Нарушение правил уборки от снега и наледи территории парка и сквера',
                 86: '★ Наледь и сосульки на кровле',
                 87: 'График движения общественного транспорта',
                 88: 'Пенсионеры и ветераны',
                 89: 'Государственные услуги',
                 90: '★ Отсутствие урн, лавочек в общественных местах и дворовой территории',
                 91: 'Детские оздоровительные лагеря',
                 92: 'Низкая температура воды/слабое давление',
                 93: 'Строительство спортивной инфраструктуры',
                 94: '★ Прорыв трубы/трубопровода',
                 95: 'Безопасность образовательного процесса',
                 96: 'Нарушение правил проведения земляных работ',
                 97: 'Хамство медицинских работников',
                 98: 'Зарплата, компенсации, поощрения, выплаты',
                 99: 'Зарплата учителей',
                 100: 'Добавить новый маршрут',
                 101: '★ Нестационарная торговля (киоски, павильоны, сезонная торговля)',
                 102: '★ Нарушение правил уборки и вывоза порубочных остатков',
                 103: 'Ненадлежащее содержание и эксплуатация МКД',
                 104: 'Перепланировка и реконструкция помещений',
                 105: 'Дополнительное образование',
                 106: 'Дистанционное образование',
                 107: 'Строительство объектов по обращению с отходами',
                 108: 'Отсутствие электричества',
                 109: 'Отсутствие лекарств в стационарах',
                 110: '★ Несанкционированные надписи, рисунки, реклама на фасаде МКД',
                 111: 'Освещение неисправно или отсутствует',
                 112: 'Непригодные для проживания жилые помещения',
                 113: '★ Протечки с кровли (системы водостока)',
                 114: 'Обустройство асфальтового покрытия парковки, внутридворового проезда, тротуара, пешеходной дорожки, въезда во двор',
                 115: 'Низкая заработная плата врачей',
                 116: 'Раздельная сортировка отходов',
                 117: 'Организация переходов, светофоров/Изменить организацию движения',
                 118: 'Состояние зданий учреждений и организаций',
                 119: 'Ремонт спортивных учреждений',
                 120: 'Памятники и объекты культурного наследия',
                 121: 'Общее впечатление',
                 122: 'Благоустройство придомовых территорий',
                 123: 'Строительство школ, детских садов',
                 124: 'Запрос на газификацию и её условия',
                 125: 'Улучшение жилищных условий',
                 126: 'Нехватка материально-технического обеспечения',
                 127: 'Работа светофора (установка, изменение режима работы, оборудование кнопкой вызова)',
                 128: 'Нехватка мест в школах',
                 129: 'Отсутствие остановочных пунктов',
                 130: 'Содержание гос. образовательных организаций',
                 131: 'Выбросы вредных веществ в атмосферу/загрязнение воздуха',
                 132: '★ Нарушение правил уборки и вывоза загрязненного снега и наледи на газоне',
                 133: 'Сертификаты и QR-коды',
                 134: '★ Ненадлежащее состояние игровых элементов на детской или спортивной площадке',
                 135: '★ Несоблюдение правил уборки проезжей части',
                 136: 'Волонтерство',
                 137: 'Проблемы с социальными картами или проездными документами',
                 138: 'Проблемы в работе горячих линий',
                 139: '★ Подтопление территории',
                 140: 'Социальная поддержка медперсонала',
                 141: 'ВУЗы и ССУЗы',
                 142: 'Строительство социальных объектов',
                 143: 'Коронавирусные ограничения',
                 144: 'Питание',
                 145: 'ЕГЭ, ОГЭ',
                 146: 'Содержание дорожных знаков/Установка новых дорожных знаков, с внесением в схему дислокации, замена старых знаков на новые',
                 147: 'Спортивные мероприятия',
                 148: 'Строительство зданий',
                 149: 'Выплаты стипендий',
                 150: 'Нарушение технологии и (или) сроков производства работ',
                 151: 'Выбросы вредных веществ в водоёмы/загрязнение водоёмов',
                 152: 'Изменение класса и количества автобусов',
                 153: 'Самоизоляция и карантин',
                 154: 'Подключение к водоснабжению',
                 155: 'Региональное имущество',
                 156: 'Изменить или отменить маршрут',
                 157: 'Пробки',
                 158: '★ Нарушение правил уборки внутридворового проезда, пешеходной дорожки',
                 159: 'Предложение дороги в план ремонта',
                 160: 'Отсутствие аптек',
                 161: 'Выброс мусора нарушителем',
                 162: 'Ремонт подъездов',
                 163: 'Санитарно-эпидемиологическое благополучие',
                 164: 'Предложения по созданию лечебных центров',
                 165: 'Проблемы с отоплением детских садов и школ',
                 166: '★ Стихийные свалки в городе/в парках/в лесу',
                 167: 'Ненадлежащее состояние фасадов нежилых зданий, объектов и ограждений',
                 168: 'Обработка и уничтожение грызунов (дератизация)',
                 169: 'Погребение и похоронное дело',
                 170: 'Другие проблемы с общедомовой системой водоотведения (канализации)',
                 171: 'Качество электроснабжения',
                 172: 'Отсутствие общественных туалетов',
                 173: 'Беженцы',
                 174: 'МФЦ "Мои документы"',
                 175: '★ Засор в общедомовой системе водоотведения (канализации)',
                 176: 'Несанкционированное ограничение движения, помехи движению, захват земли в полосе отвода автодорог',
                 177: 'Архитектура города',
                 178: 'Социальная поддержка учителей',
                 179: 'Установка ограждений, препятствующих въезду на тротуар, газон на дворовой территории МКД',
                 180: 'Лифт неисправен или отключен',
                 181: 'Самовольная установка ограждений на территории общего пользования',
                 182: 'Кадровые перестановки',
                 183: 'Некачественно выполненный ремонт дорог',
                 184: 'Здравоохранение прочее',
                 185: '★ Повреждение опор ЛЭП',
                 186: 'Сроки газификации',
                 187: 'Стоимость, оплата и возврат средств на газификацию',
                 188: '★ Нарушение правил уборки от снега и наледи детской игровой и спортивной площадки',
                 189: 'Залитие квартиры',
                 190: 'Неисправность выступающих конструкций: балконов, козырьков, эркеров, карнизов входных крылец и т. п.',
                 191: 'Соц.обслуживание прочее',
                 192: 'Устройство в ДОУ',
                 193: '★ Парковка на газонах',
                 194: 'Матери-одиночки, подростки'}

In [23]:
text_to_predict = "Добрый день. Подскажите пожалуйста по такому вопросу. Подала на выплату с 3-7. Есть старший сын студент очник. Предоставила в соц защиту справку о стипендии в электронном варианте. Соц.защита требует в бумажном виде . Сказали , что будет отказ. Дети на практике и взять справку на руки не предоставляется возможным. В прошлом году сама соц.защита предложила в электронном варианте справку послать, а в этом нужна бумага. Правомерно ли поступают в нашей соц.защите г. Чусового? Спасибо."
text_to_predict

'Добрый день. Подскажите пожалуйста по такому вопросу. Подала на выплату с 3-7. Есть старший сын студент очник. Предоставила в соц защиту справку о стипендии в электронном варианте. Соц.защита требует в бумажном виде . Сказали , что будет отказ. Дети на практике и взять справку на руки не предоставляется возможным. В прошлом году сама соц.защита предложила в электронном варианте справку послать, а в этом нужна бумага. Правомерно ли поступают в нашей соц.защите г. Чусового? Спасибо.'

In [24]:
predicted_class, confidence = predict_class(text_to_predict, large_tokenizer, large_label_model, classes_large)

print(f"Предсказанный класс: {predicted_class}, Вероятность: {confidence}")

Предсказанный класс: Социальное обслуживание и защита, Вероятность: 0.9984623193740845


In [25]:
text_to_predict = predicted_class + ': ' + text_to_predict
predicted_class, confidence = predict_class(text_to_predict, small_tokenizer, small_label_model, classes_small)

print(f"Предсказанный класс: {predicted_class}, Вероятность: {confidence}")

Предсказанный класс: Дети и многодетные семьи, Вероятность: 0.9818469882011414
